# 데이터 전처리

# Garbage In, Garbage Out

# 사이킷런을 이용한 ML 학습하기 전에 데이터에 대해 처리할 기본사항

## 1. 결손값 (NA, NaN, NULL) 값은 허용되지 않는다.

    * 대체값을 선정해서 치환

    * Drop

## 2. 사이킷런의 머신러닝 알고리즘은 문자열을 입력값으로 받지 않는다.

    * 인코딩

    * Drop



In [ ]:
# 모듈 로딩
import seaborn as sns
import pandas as pd

## 결측치 (누락 데이터) 처리

In [ ]:
# 데이터 로딩
df = sns.load_dataset('titanic')
df.head()

### **[누락데이터 확인]**

In [ ]:
# 기본 요약 정보 확인
df.info()

In [ ]:
# deck 컬럼에 NaN 개수
nan_deck = df['deck'].value_counts(dropna=False)    # value_counts : default가 NaN을 제외하고 갯수를 구하도록 설계
nan_deck

In [ ]:
# isnull() - 직접적인 방법 : 누락데이면 True, 아니면 False
df.head().isnull()

In [ ]:
# notnull() - 간접적인 방법 : 유효한 데이터면 True, 누락데이터면 False
df.tail().notnull()

In [ ]:
# 직접적인 방법으로 누락데이터 갯수 구하기
df.head().isnull().sum(axis=0)     # True : 1, False : 0으로 인정되므로 연산이 가능

In [ ]:
# [실습] 전체 데이터 각 컬럼의 누락데이터 갯수를 구해보세요


### **[누락데이터 삭제]**

* 컬럼이 데이터 관계상 상관관계가 별로 없고, 누락데이터가 너무 많을 경우

* 이 컬럼을 삭제하는 것이 분석(예측)에 더 의미가 있다고 판단 되는 아주 특수한 경우

In [ ]:
df_thresh = df.dropna(axis=1, thresh=500)            # thresh : NaN 최소 기준치
df_thresh.columns

In [ ]:
# age 컬럼에 나이 데이터가 없는 행을 삭제
df_age = df.dropna(subset=['age'], how='any', axis=0)
print(len(df_age))

### **[누락 데이터 치환(대치)]**

* 보통 대체할 값으로는 데이터의 분포와 특성을 잘 나타낼 수 있는 값 : 평균값, 중앙값, 최빈값 등

In [ ]:
df.head()

In [ ]:
# age 컬럼 확인
df.age.head(10)

* 평균값으로 대치

In [ ]:
mean_age = df.age.mean(axis=0)
df.age.fillna(mean_age, inplace=True)

In [ ]:
df.age.isnull().sum(axis=0)

* 최빈값으로 대치

In [ ]:
df['embark_town'][825:835]

In [ ]:
# 승선도시 중에서 가장 많이 출현한 값으로 대치
most_freq = df['embark_town'].value_counts(dropna=True).idxmax()

most_freq

In [ ]:
df['embark_town'].fillna(most_freq, inplace=True)
df['embark_town'][825:835]

* 이전 값으로 대치

* 데이터셋의 특성 상 서로 이웃하고 있는 데이터끼리는 유사성을 가질 가능성이 높기 때문

In [ ]:
df['embarked'][825:835]

In [ ]:
# embarked 컬럼의 NaN값을 찾아서 바로 이전행 값으로 치환
df['embarked'].fillna(method='ffill', inplace=True)
df['embarked'][825:835]

In [ ]:
df.info()

In [ ]:
'''
[참고] 누락데이터가 NaN으로 표시되지 않은 경우
예를 들어서 숫자 0 또는 문자 '-', '?', '=" 값으로 입력되어 있는 데이터셋들이 있음.

# 해결방법은 위 NaN이 아닌 기호나 숫자를 NaN으로 바꾼 후 NaN 처리 함수를 사용하면 더 생산성 높게 처리 가능.
ex) df.replace('-', np.nan, inplace=True)

'''

## 중복데이터 처리

### **[중복 데이터 확인]**

In [ ]:
# 중복 데이터 생성
df = pd.DataFrame({
    'c1' : ['a', 'a', 'b', 'a', 'b'],
    'c2' : [1, 1, 1, 2, 2],
    'c3' : [1, 1, 2, 2, 2]
})

df

In [ ]:
# duplicated : 중복값이면 True / 아니면 False 반환

# 전체 행에서 각 컬럼별 중복값이 모두 일치하는 행 찾기
df_dup = df.duplicated()

df_dup

In [ ]:
# 특정 컬럼별 중복값 찾기
col_dup = df.c2.duplicated()
col_dup

### **[중복 데이터 제거]**

In [ ]:
df

In [ ]:
# 전체 행에서 중복되는 행을 제거
df2 = df.drop_duplicates()
df2

In [ ]:
# 특정 컬럼을 기준으로 중복되는 행을 제거
df3 = df.drop_duplicates(subset='c2')
df3

In [ ]:
# 기준이 되는 컬럼을 여러 개 사용 가능
df4 = df.drop_duplicates(subset=['c2', 'c3'])
df4

In [ ]:
df

## 데이터 표준화

* 데이터가 잘 정리된 것 처럼 보여도 서로 다른 단위가 섞여 있거나 같은 대상을 다른 형식으로 표현하는 경우가 의외로 많다.

* 동일한 대상을 표현하는 방법에 차이가 있으면 분석(예측) 정확도는 현저히 낮아진다.

* 데이터 포맷을 일관성 있게 표준화 하는 작업이 필요하다.

* 데이터의 출처 또는 데이터에 대한 설명이 반드시 필요!!

### **[단위 환산]**

In [ ]:
'''
   1.  mpg:               연비                          continuous
    2. cylinders:         실린더수                        multi-valued discrete
    3. displacement:      배기량                           continuous
    4. horsepower:       마력                              continuous
    5. weight:           차중                               continuous
    6. acceleration:     가속                                 continuous
    7. model year:       생산년도                        multi-valued discrete
    8. origin:           생산지역                     multi-valued discrete (1 : USA, 2 : EU, 3 : JPN)
    9. car name:         자동차명                    string (unique for each instance)
'''

In [ ]:
feature_name = ['mpg','cylinders', 'displacement', 'horsepower', 'weight',
                'acceleration', 'model year', 'origin', 'car name']

In [ ]:
df = pd.read_csv('./auto-mpg.csv', header=None)
df.columns = feature_name
df.head()

In [ ]:
# mpg (mile per gallon) - kpl (killometer per liter)로 변환 (mpg to kpl = 0.425)
mpg_to_kpl = 1.60934 / 3.75841

# 데이터셋에 kpl 열을 추가
df2 = df.copy()

df2['kpl'] = (df2.mpg * mpg_to_kpl)

df2.head(3)

### **[자료형 변환]**

In [ ]:
df.head()

In [ ]:
df3 = df.copy()

df3.head()

In [ ]:
# 전체 기본 정보
df3.info()

In [ ]:
# horsepower 컬럼의 고유값 확인
df3.horsepower.unique()

In [ ]:
# '?' 관측값 처리 : NaN으로 바꾸고 NaN 처리
import numpy as np

df3.horsepower.replace('?', np.nan, inplace=True)

df3.horsepower.unique()

In [ ]:
# '?' - nan - 삭제 처리
df3.dropna(subset='horsepower', axis=0, inplace=True)

df3.horsepower.unique()

In [ ]:
# 현재 object인 horsepower 컬럼 자료형을 수치형(실수형)으로 변환
df3.horsepower = df3.horsepower.astype('float')

df3.horsepower.dtypes

In [ ]:
df3.horsepower.unique()

## 데이터 정규화

* 각 컬럼(특징, 변수)에 들어있는 숫자데이터의 상대적 크기 차이는 특정 분석(알고리즘) 방법에 따라서 분석 결과를 달라지게 할 수 있다.

* 상대적 크기 차이를 제거하기 위해 데이터 값을 동일한 크기 기준으로 나누는 작업을 정규화(normalization) 이라고 한다.

* 보통 정규화 과정을 거친 데이터의 범위는
  0 ~ 1 사이 또는 -1 ~ 1 사이가 된다.

----

* [참고] 경우에 따라서는 특정 알고리즘이 상대적 크기 차이를 가정하고 생성되어 있다면 상대적 크기 차이를 갖도록 데이터를 제공하는 것이 더 좋은 결과를 얻을 수 있다.

### **[정규화 1. 해당 컬럼의 최대값(의 절대값)으로 나누기]**

In [ ]:
df4 = df3.copy()

In [ ]:
# 기술통계
df3.horsepower.describe()

In [ ]:
# 정규화1 처리
df3.horsepower = df3.horsepower / abs(df3.horsepower.max())

df3.head()

In [ ]:
df3.horsepower.describe()

### **[정규화2. 각 컬럼 데이터 중에서 최대값과 최소값을 뺀 값으로 나누는 방법]**

In [ ]:
'''
# 각 컬럼 데이터에서 해당 열의 최소값을 뺀 값을 분자,
# 해당 열의 최대값과 최소값의 차이를 분모,
# 가장 큰 값은 역시 1이 됨.
'''

In [ ]:
numerator = df4.horsepower - df4.horsepower.min()   # 분자
denominator = df4.horsepower.max() - df4.horsepower.min()   # 분모

df4.horsepower = numerator / denominator

df4.horsepower.head()

In [ ]:
df4.horsepower.describe()

## 범주형(카테고리) 데이터 처리

* 알고리즘에 따라서는 연속데이터를 그대로 사용하기 보다는 일정한 구간(bin)으로 나눠서 분석(학습) 하는 것이 효율적인 경우가 있음.

* 가격, 비용, 효율 등의 의미를 가지고 있는 연속적인 값은 일정한 수준이나 정도를 가지고 있는 이산적인 값으로 나타내어서 수준의 차이를 드러내는 방식으로 구현하면 더 효율적인 경우가 있다

* 이런 과정을 구간 분할 (binning)이라고 함.

      pandas : cut()

### **[구간 분할]**

In [ ]:
df2 = df.copy()

In [ ]:
df2.horsepower.replace('?', np.nan, inplace=True)
df2.dropna(subset='horsepower', axis=0, inplace=True)
df2.horsepower = df2.horsepower.astype('float')
df2.horsepower.unique()

In [ ]:
import numpy as np

In [ ]:
# 3개의 구간으로 나누어서 표기
# np.histogram()

count, bin_dividers = np.histogram(df2.horsepower, bins=3)

print(count)        # 구간별 데이터의 수
print('-'*50)
print(bin_dividers)    # 구간의 경계값들

In [ ]:
# 나눠지는 경계로 구간명을 지정
bin_names = ['저출력', '보통출력', '고출력']

In [ ]:
# 각 데이터를 각 구간에 할당 처리
df2['hp_bins'] = pd.cut(x=df2['horsepower'],     # 원 데이터 배열
                        bins = bin_dividers,          # 경계 값 리스트
                        labels=bin_names,             # 구간의 이름
                        include_lowest=True          # 첫 경계값 포함 여부
                        )

df2[['horsepower', 'hp_bins']].head(15)

In [ ]:
# 전체 데이터 확인
df2.head(10)

### **[더미 변수]**

* 위 hp_bins 범주형데이터는 문자열이므로 컴퓨터(모델)이 인식할 수 없는 형태

* 특정 알고리즘은 더 높은 숫자값에 가중치를 부여하는 경우가 있으므로 주의가 필요.

* pandas는 더미변수(숫자 0 또는 1로 표현하는 값) 라는 것을 이용.

* 어떤 특성이 있는지 여부만 표현. 특성이 있으면 1, 없으면 0.

* 이 개념을 머신러닝에서는 one-hot encoding


        pandas : get_dummis()



In [ ]:
# hp_bins 컬럼(범주형 데이터)를 더미 변수로 변환
horsepower_dummies = pd.get_dummies(df2['hp_bins'], dtype='int')

horsepower_dummies.head(15)

In [ ]:
pd.__version__



---



## 데이터 인코딩

### **[레이블 인코딩]**

* 카테고리 피처를 코드형 숫자값으로 변환하는 방식

In [ ]:
items = ['TV', '냉장고', '전자렌지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encoder 객체를 생성, fit() - transform() 으로 label encoding 수행
encoder = LabelEncoder()

encoder.fit(items)

labels = encoder.transform(items)

print('인코딩 변환갑 : ', labels)

In [ ]:
print('인코딩 클래스 : ', encoder.classes_)

#items = ['TV', '냉장고', '전자렌지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

In [ ]:
# 원본 정보를 가지고 있으므로 디코딩 가능
print('디코딩 클래스 : ', encoder.inverse_transform([4, 5, 2, 0, 1, 1, 3, 3]))

In [ ]:
'''
[주의]
# 레이블 인코딩은 labeldl 숫자(정수)가 1씩 증가되는 형태로 변환되는 특성이 있음
# 연속형 숫자로 이뤄지는 특성에 영향을 받는 모델일 경우에는 label encoding이 문제가 될 수 있다.

# => one-hot encoding

'''

### **[원-핫(one-hot) 인코딩]**

* 고유값에만 해당하는 컬럼에만 1을 표시하고 나머지는 0을 표시하는 방식

* 유의사항

  1. 모든 문자열의 값은 숫자형 값이어야 한다.

  2. 입력값으로 2차원의 데이터이어야 한다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

items = ['TV', '냉장고', '전자렌지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

# 숫자값 변환을 위해 LabelEncoder로 변환이 먼저
encoder = LabelEncoder()
labels = encoder.fit_transform(items)
print(f'인코딩 변환값 : {labels}')

In [ ]:
# 인코딩 변환값을 2차원 데이터로 변환
labels = labels.reshape(-1, 1)
labels

In [ ]:
# 원-핫 인코딩을 적용
oh_encoder = OneHotEncoder()
oh_encoder.fit(labels)
oh_labels = oh_encoder.transform(labels)

print('------- 원 - 핫 인코딩 관측값 -------')
print(oh_labels.toarray())
print(f'원-핫 인코딩 데이터 차원 : ', oh_labels.shape)

In [ ]:
# 같은 결과를 pandas로 얻어낼 수 있음

df = pd.DataFrame({'items': ['TV', '냉장고', '전자렌지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']})

df

In [ ]:
pd.get_dummies(df, dtype='int')

## 피처스케일링(Feature Scaling)

* 서로 다른 변수의 값 범위를 일정한 수준으로 맞추는 작업

    * 피처스케일링의 대표적 방법 : 표준화(Standardization), 정규화(Normalization)

In [ ]:
'''
# 표준화 : 데이텟의 피처 각각이 평균이 0이고 분산이 1인 가우시안 정규분포를 가진 값으로 변환하는 것.
# 정규화 : 서로 다른 피처 크기를 통일시키기 위해 그 크기를 변환해주는 것

# => 사이킷런은 이런 작업을 통칭해서 피처스케일링

'''

### **[StandardScaler]**

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df.head()

In [ ]:
print('--- feature 들의 평균값 ----')
print(iris_df.mean())
print('--- feauter 들의 분산값 ----')
print(iris_df.var())

In [ ]:
from sklearn.preprocessing import StandardScaler

# 객체 생성
scaler = StandardScaler()

# 데이터셋 변환 : fit(), transform()
scaler.fit(iris_df)

iris_scaled = scaler.transform(iris_df)

iris_scaled

In [ ]:
# 피처스케일링 결과과 ndarray 객체로 리턴된다
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)

print('--- feature 들의 평균값 ----')
print(iris_df_scaled.mean())
print('--- feauter 들의 분산값 ----')
print(iris_df_scaled.var())

### **[MinMaxScaler]**

* 데이터 값을 0과 1사이의 범위 값으로 변환 (음수값이 있으면 -1부터 1값으로 변환)

* 데이터 분포가 가우시안 분포가 아닐 경우에 적용해 볼 수 있음.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 객체 생성
scaler = MinMaxScaler()

# fit(), transform()
scaler.fit(iris_df)

iris_scaled = scaler.transform(iris_df)

In [ ]:
# 피처스케일링 결과과 ndarray 객체로 리턴된다
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)

print('--- feature 들의 최소값 ----')
print(iris_df_scaled.min())
print('--- feauter 들의 최대값 ----')
print(iris_df_scaled.max())

### **[유의사항]**

* Scaler를 이용해서 학습데이터와 테스트데이터에 fit(), transform(), fit_transform() 적용시

* 학습데이터로 fit()을 수행한 스케일링 기준 정보를 그대로 테스트 데이터에 적용해야 한다.


In [ ]:
# 문제 상황
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [ ]:
# 학습데이터는 0~10까지, 테스트데이터 : 0~5 값을 갖는 데이터 세트
# fit(), transform() 2차원 이상의 데이터 형태만 가능함
train_array = np.arange(0, 11).reshape(-1, 1)
test_array = np.arange(0, 6).reshape(-1, 1)

In [ ]:
# 피처 스케일링 (train data) 적용

# 객체 호출
scaler = MinMaxScaler()

# fit() - 기준 적용 (최소값을 0, 최대값을 10으로 설정)
scaler.fit(train_array)

# transform() - 1/10 scale 적용
train_scaled = scaler.transform(train_array)

# 데이터 확인
print('원본 train_array 데이터 값 : ', np.round(train_array.reshape(-1), 2))
print('scaled 된 train_array 데이터 값 : ', np.round(train_scaled.reshape(-1), 2))

In [ ]:
# 피처 스케일링 (test data) 적용

# 객체 호출
# fit()   - 기준 (최소값을 0, 최대값을 5)  => 새로운 기준 정보가 생성되었다.
scaler.fit(test_array)

# transform - 1/5 scale
test_scaled = scaler.transform(test_array)

# 데이터 확인
print('원본 test_array 데이터 값 : ', np.round(test_array.reshape(-1), 2))
print('scaled 된 test_array 데이터 값 : ', np.round(test_scaled.reshape(-1), 2))

In [ ]:
# 올바른 방법
scaler = MinMaxScaler()

# fit() - 기준 적용 (최소값을 0, 최대값을 10으로 설정)
scaler.fit(train_array)

# transform() - 1/10 scale 적용
train_scaled = scaler.transform(train_array)

# 데이터 확인
print('원본 train_array 데이터 값 : ', np.round(train_array.reshape(-1), 2))
print('scaled 된 train_array 데이터 값 : ', np.round(train_scaled.reshape(-1), 2))

# 테스트 데이터의 scale 변환은 fit() 호출없이 train 데이터의 기준을 적용해서 바로 transform()만 수행
test_scaled = scaler.transform(test_array)

# 데이터 확인
print('원본 test_array 데이터 값 : ', np.round(test_array.reshape(-1), 2))
print('scaled 된 test_array 데이터 값 : ', np.round(test_scaled.reshape(-1), 2))